# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**: Antonia Hoerburger

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://aba0da925ac6:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/26 13:07:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from antoniahorburger.spark_utils import SparkUtils

In [3]:
!pwd

/opt/spark/work-dir/labs/lab04


In [4]:
!ls /opt/spark/work-dir/data/car_service/

agencies  brands  cars	customers  rentals


In [5]:
agencies_schema = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])
agencies_df = spark.read.option("header", True).schema(agencies_schema).csv("/opt/spark/work-dir/data/car_service/agencies")
agencies_df.show()

[Stage 0:>                                                          (0 + 1) / 1]

+---------+--------------------+
|agency_id|         agency_info|
+---------+--------------------+
|        1|{'agency_name': '...|
|        2|{'agency_name': '...|
|        3|{'agency_name': '...|
|        4|{'agency_name': '...|
|        5|{'agency_name': '...|
+---------+--------------------+



In [6]:
from pyspark.sql.functions import get_json_object

agencies_schema = SparkUtils.generate_schema([("agency_id", "string"), ("agency_info", "string")])
cars_schema = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])
customers_schema = SparkUtils.generate_schema([("customer_id", "int"), ("customer_info", "string")])
rental_cars_schema = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])

base_path = "/opt/spark/work-dir/data/"

agencies_df = spark.read \
                .schema(agencies_schema) \
                .option("header", "true") \
                .csv(base_path + "car_service/agencies")
agencies_df = agencies_df.withColumn("agency_name", get_json_object(agencies_df.agency_info, "$.agency_name"))


cars_df = spark.read.option("header", "true").schema(cars_schema).csv(base_path + "car_service/cars")

brands_df = spark.read.option("header", "true").schema(cars_schema).csv(base_path + "car_service/brands")

customers_df = spark.read.option("header", "true").schema(customers_schema).csv(base_path + "car_service/customers")

rental_cars_df = spark.read.option("header", "true").schema(rental_cars_schema).csv(base_path + "car_service/rentals")

In [7]:
agencies_df = agencies_df.withColumn("agency_name", get_json_object(agencies_df.agency_info, "$.agency_name")).drop("agency_info")
cars_df = cars_df.withColumn("car_name", get_json_object(cars_df.car_info, "$.car_name")).drop("car_info")
customers_df = customers_df.withColumn("customer_name", get_json_object(customers_df.customer_info, "$.customer_name")).drop("customer_info")

rental_cars_df = rental_cars_df.withColumn("car_id", get_json_object(rental_cars_df.rental_info, '$.car_id')) \
                            .withColumn("customer_id", get_json_object(rental_cars_df.rental_info, '$.customer_id')) \
                            .withColumn("agency_id", get_json_object(rental_cars_df.rental_info, '$.agency_id')) \
                            .drop("rental_info")

In [8]:
rental_cars_df = rental_cars_df.join(cars_df, on="car_id", how="left") \
                                .join(customers_df, on="customer_id", how="left") \
                                .join(agencies_df, on="agency_id", how="left") \
                                .drop("agency_id", "customer_id", "car_id")

rental_cars_df.show(5)

+---------+--------------------+---------------+-------------+
|rental_id|            car_name|  customer_name|  agency_name|
+---------+--------------------+---------------+-------------+
|    11891|Wallace-Carlson M...| Margaret Jones|  NYC Rentals|
|    11892|Grimes-Green Model 8|Albert Williams|LA Car Rental|
|    11893|Stewart-Allen Mod...|  Caleb Fleming|      SF Cars|
|    11894|  Campos PLC Model 4|  Andrew Butler|  NYC Rentals|
|    11895|  Wagner LLC Model 1|  Kristin Potts|      SF Cars|
+---------+--------------------+---------------+-------------+
only showing top 5 rows


In [10]:
!pwd

/opt/spark/work-dir/labs/lab04


In [29]:
!du -sh /opt/spark/work-dir/data/car_service/rentals/

1.1M	/opt/spark/work-dir/data/car_service/rentals/


In [30]:
rental_cars_df.write \
            .mode("overwrite") \
            .parquet(base_path + "car_service/output/parquet")

In [31]:
!ls /opt/spark/work-dir/data/car_service/output/parquet/

part-00000-d4a4f969-0fa3-491b-b6b5-06393114a66e-c000.snappy.parquet  _SUCCESS
part-00001-d4a4f969-0fa3-491b-b6b5-06393114a66e-c000.snappy.parquet


In [32]:
!du -sh /opt/spark/work-dir/data/car_service/output/parquet/

124K	/opt/spark/work-dir/data/car_service/output/parquet/


In [33]:
rental_cars_df.write \
            .partitionBy("agency_name") \
            .mode("overwrite") \
            .parquet(base_path + "car_service/output/parquet")

In [34]:
!ls /opt/spark/work-dir/data/car_service/output/parquet/

'agency_name=LA Car Rental'  'agency_name=SF Cars'	  _SUCCESS
'agency_name=NYC Rentals'    'agency_name=Zapopan Auto'


In [35]:
!du -sh /opt/spark/work-dir/data/car_service/output/parquet/

144K	/opt/spark/work-dir/data/car_service/output/parquet/


In [36]:
rental_cars_df.write \
            .partitionBy("agency_name") \
            .mode("overwrite") \
            .option("header", True) \
            .csv(base_path + "car_service/output/csv")

In [37]:
!du -sh /opt/spark/work-dir/data/car_service/output/csv

856K	/opt/spark/work-dir/data/car_service/output/csv
